In [1]:
import carla

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

In [5]:
vehicles = world.get_actors().filter('vehicle.*')
print(f"Number of vehicles: {len(vehicles)}")


Number of vehicles: 30


In [7]:
def attach_collision_sensor(vehicle, vehicle_data):
    blueprint_library = world.get_blueprint_library()
    collision_sensor_bp = blueprint_library.find('sensor.other.collision')
    collision_sensor = world.spawn_actor(
        collision_sensor_bp,
        carla.Transform(),
        attach_to=vehicle
    )

    def on_collision(event):
        vehicle_id = vehicle.id
        other_actor = event.other_actor
        print(f"Collision detected for Vehicle {vehicle_id} with {other_actor.type_id}")
        if vehicle_id in vehicle_data:
            vehicle_data[vehicle_id]["collision_count"] += 1

    collision_sensor.listen(on_collision)
    return collision_sensor

# Get all vehicles spawned by generate_traffic.py
vehicles = world.get_actors().filter('vehicle.*')

# Attach sensors
vehicle_data = {}
collision_sensors = []
for vehicle in vehicles:
    vehicle_data[vehicle.id] = {
        "speeds": [],
        "distance_traveled": 0.0,
        "last_location": vehicle.get_location(),
        "collision_count": 0
    }
    sensor = attach_collision_sensor(vehicle, vehicle_data)
    collision_sensors.append(sensor)


In [12]:
import time

start_time = time.time()


while time.time() - start_time < 60:  # Run for 1 minute
    world.tick()  # Advance simulation
    for vehicle in vehicles:
        velocity = vehicle.get_velocity()
        speed = 3.6 * ((velocity.x**2 + velocity.y**2 + velocity.z**2) ** 0.5)  # Convert to km/h
        vehicle_data[vehicle.id]["speeds"].append(speed)

        if vehicle.id not in vehicle_data:
            vehicle_data[vehicle.id] = {
                "speeds": [],
                "distance_traveled": 0.0,
                "last_location": vehicle.get_location(),
                "collision_count": 0
            }

        # Update speed
        vehicle_data[vehicle.id]["speeds"].append(speed)

        # Calculate distance traveled
        current_location = vehicle.get_location()
        last_location = vehicle_data[vehicle.id]["last_location"]
        distance = ((current_location.x - last_location.x)**2 +
                    (current_location.y - last_location.y)**2 +
                    (current_location.z - last_location.z)**2) ** 0.5
        vehicle_data[vehicle.id]["distance_traveled"] += distance
        vehicle_data[vehicle.id]["last_location"] = current_location


Collision detected for Vehicle 24 with static.vegetation
Collision detected for Vehicle 53 with static.static
Collision detected for Vehicle 37 with static.sidewalk
Collision detected for Vehicle 47 with static.vegetation
Collision detected for Vehicle 52 with static.vegetation
Collision detected for Vehicle 36 with static.pole
Collision detected for Vehicle 52 with static.pole
Collision detected for Vehicle 49 with static.car
Collision detected for Vehicle 51 with static.static
Collision detected for Vehicle 38 with static.vegetation
Collision detected for Vehicle 34 with static.building
Collision detected for Vehicle 52 with static.vegetation
Collision detected for Vehicle 32 with vehicle.ford.ambulance
Collision detected for Vehicle 25 with static.static
Collision detected for Vehicle 42 with static.pole
Collision detected for Vehicle 33 with static.building
Collision detected for Vehicle 28 with vehicle.mercedes.sprinter
Collision detected for Vehicle 36 with static.pole
Collision 

In [14]:
import pandas as pd

processed_data = []
for vehicle_id, data in vehicle_data.items():
    avg_speed = sum(data["speeds"]) / len(data["speeds"]) if data["speeds"] else 0
    max_speed = max(data["speeds"], default=0)
    processed_data.append({
        "Vehicle ID": vehicle_id,
        "Average Speed (km/h)": avg_speed,
        "Max Speed (km/h)": max_speed,
        "Distance Traveled (m)": data["distance_traveled"],
        "Collisions": data["collision_count"]
    })

df = pd.DataFrame(processed_data)
df.to_csv("vehicle_metrics.csv", index=False)
